In [ ]:
import pandas as pd
data = pd.read_csv('/content/dataset.csv', sep=",")
data

,username,clean,gpt
0,muldirun,masuk ke dalam daftar blunder ganjar,NEGATIVE
1,Ganjar_Mahfud03,psi singkatannya partai seputaran ibukota pend...,NEGATIVE
2,WinnerWave_,asik benar menyimak obrolan ini kalau sampai k...,POSITIVE
3,fadsdwo,mohon maaf pak ganjar di grand final master ch...,NEGATIVE
4,BangPakat,kaos dari tuhan sudah selesai bersama ganjar m...,POSITIVE
...,...,...,...
5572,B3doel___,tpn tim pemenangan nasional sebut ganjar mahfu...,POSITIVE
5573,siticeriaselalu,pantas saja generasi muda nu generasi mudanu d...,POSITIVE
5574,mikuroQ,ganjar pranowo memperhatikan kurangnya fasilit...,POSITIVE
5575,seruanhl,program juta lapangan kerja baru merupakan tin...,POSITIVE


# PREPRO


In [ ]:
!pip install indoNLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.0 MB/s eta 0:00:00


In [ ]:
from indoNLP.preprocessing import replace_slang
from indoNLP.preprocessing import remove_url
from indoNLP.preprocessing import replace_word_elongation

In [ ]:
import re
def prepro(text):
    text = re.sub(r'[#@]\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'http\S+', '', text)
    text = text.lower()
    text = re.sub(r' +', ' ', text).strip()
    return text

In [ ]:
# Assuming your DataFrame is named df
data.rename(columns={'processed': 'clean'}, inplace=True)
data

,username,clean,gpt
0,muldirun,masuk ke dalam daftar blunder ganjar,negatif
1,Ganjar_Mahfud03,psi singkatannya partai seputaran ibukota pend...,negatif
2,WinnerWave_,asik benar menyimak obrolan ini kalau sampai k...,positif
3,fadsdwo,mohon maaf pak ganjar di grand final master ch...,negatif
4,BangPakat,kaos dari tuhan sudah selesai bersama ganjar m...,positif
...,...,...,...
5572,B3doel___,tpn tim pemenangan nasional sebut ganjar mahfu...,positif
5573,siticeriaselalu,pantas saja generasi muda nu generasi mudanu d...,positif
5574,mikuroQ,ganjar pranowo memperhatikan kurangnya fasilit...,positif
5575,seruanhl,program juta lapangan kerja baru merupakan tin...,positif


# FINE TUNED

In [ ]:
from transformers import TFAutoModelForMaskedLM

model_checkpoint = "w11wo/indonesian-roberta-base-posp-tagger"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForMaskedLM: ['classifier.weight', 'classifier.bias', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForMaskedLM were not initialized from the PyTorch model and are newly initialized: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and 

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
tokenizer(list(data['clean'].iloc[:2]))

{'input_ids': [[1119, 316, 395, 1904, 48490, 382, 5961], [273, 376, 13114, 303, 3817, 33283, 10802, 4533, 20631, 11224, 3817, 46997, 11224, 1021, 663, 39260, 382, 560, 5211, 5135, 1296, 4613, 1354, 9531, 1932, 44158, 14063, 10867, 11946, 265, 3795, 504, 46226, 541, 1563, 9531, 1932, 14063, 2582, 399, 672, 2141, 4613, 271, 5786, 382, 560, 5211]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
!pip install datasets -qq

In [ ]:
from sklearn.model_selection import train_test_split

data.columns = ["username","clean", "gpt"]
data = data.dropna()
train, test = train_test_split(data, test_size=0.3, random_state=42)
train.shape, test.shape

((3903, 3), (1674, 3))

In [ ]:
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files={"train": "train.csv", "test":"test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
import numpy as np
import tensorflow as tf

def tokenize_function(data):
    result = tokenizer(data["clean"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=["username", "clean", "gpt"]
)
tokenized_datasets

Map:   0%|          | 0/3903 [00:00<?, ? examples/s]

Map:   0%|          | 0/1674 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 3903
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 1674
    })
})

In [ ]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 27'
'>>> Review 1 length: 54'
'>>> Review 2 length: 37'


In [ ]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 118'


In [ ]:
chunk_size = 128
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 118'


In [ ]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/3903 [00:00<?, ? examples/s]

Map:   0%|          | 0/1674 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 435
    })
})

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' tangkap gibran mau menghindari debat cawapres calon wakil presiden kpu nya disuruh rubah aturan mainnya lanjut paman gibran arif budi sulistyo tersangkut kasus suap dirjen pajak tahun waktu itu pt ekp memberikan suap senilai sama kepada dirjen pajak agar dibantu dalam urusan pajak apakah jokowi mengintervensi kasus ini juga dpr ri hamdanzoelva jokowipantasdimakzulkan hari pertama kampanye prabowo rapat bersama jokowi jam lebih di istana bogor masih seputar rumput jis bahwa rumput jis yang dirancang pak anies pada jakarta internasional stadium memakai rumput hybrid yg banyak dipakai stadion internasional di eropa jadi mahakarya jis itu gak kaleng kaleng dibuatnya sangat'

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["labels"])

' tangkap gibran mau menghindari debat cawapres calon wakil presiden kpu nya disuruh rubah aturan mainnya lanjut paman gibran arif budi sulistyo tersangkut kasus suap dirjen pajak tahun waktu itu pt ekp memberikan suap senilai sama kepada dirjen pajak agar dibantu dalam urusan pajak apakah jokowi mengintervensi kasus ini juga dpr ri hamdanzoelva jokowipantasdimakzulkan hari pertama kampanye prabowo rapat bersama jokowi jam lebih di istana bogor masih seputar rumput jis bahwa rumput jis yang dirancang pak anies pada jakarta internasional stadium memakai rumput hybrid yg banyak dipakai stadion internasional di eropa jadi mahakarya jis itu gak kaleng kaleng dibuatnya sangat'

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>>  tidak ada tempat lagi dihati<mask> jokowi dan keluarganyayg<mask> berkhianat<mask> dan ibu Moskow shame aku semakin mantab dukung ganjar<mask>fud buat yang mau serang anies pakai konvensi partai demokrat<mask>en sadar enggak bakalreskrim demokrat<mask> pak surabaya lho apa hasil<mask> bagaimana nasib pak dahlan is<mask> yang menang konvensi diusung<mask><mask><mask> presidenenggak bagaimana posisi demokrat dip<mask>pres pemilihan presiden<mask> ya<mask> tab<mask> konstitusi sudah khianati partai sudah ingkari omongan sendiri tapi hasilnya kalah ganjar<mask> semangat survei<mask> charta polit<mask> ganjar mahfud prabowo<mask>ibran<mask>atuhi tambang ilegal relawan prabowo gibran di'

'>>> <mask><mask>ibran mau menghindari debat cawapres calon wakil presiden kpu nya disuruh rubah aturan mainnya lanjut paman gibran arif budi sulistyo tersangkut kasus suap<mask><mask> pajak<mask> waktu itu pt ekp memberikan<mask> senilai<mask> kepada dirjen treat agar dibantu dalam urusan pajak apak

In [ ]:
import collections
import numpy as np

from transformers.data.data_collator import tf_default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return tf_default_data_collator(features)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>>  tidak ada tempat lagi<mask> untuk jokowi dan keluarganyayg telah berkhianat pada<mask> ibu mega<mask><mask> aku semakin mantab dukung<mask><mask> mahfud buat<mask> mau serang anies pakai<mask> partai demokrat kelen sadar<mask><mask> mempermalukan demokrat dan pak surabaya lho<mask> hasil konvensi bagaimana nasib<mask> dahlan iskan yang menang konvensi diusung jadi<mask> calon presidenenggak<mask> posisi<mask><mask> dipilpres pemilihan presiden<mask> ya sudah tabrak konstitusi sudah khianati partai<mask> ingkari omongan sendiri tapi hasilnya kalah ganjar presiden<mask> survei terbaru charta<mask><mask> ganjar mahfud prabowo gibran amin kasus tambang ilegal relawan prabowo gibran di'

'>>>  tangkap<mask><mask><mask><mask> menghindari debat cawapres calon wakil presiden kpu nya disuruh rubah<mask><mask> lanjut paman gibran arif budi sulistyo tersangkut kasus suap dirjen pajak tahun waktu itu pt ekp memberikan suap senilai sama kepada dirjen pajak agar dibantu dalam urusan pajak apa

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 435
    })
})

In [ ]:
tf_train_dataset = model.prepare_tf_dataset(
    lm_datasets["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)

tf_eval_dataset = model.prepare_tf_dataset(
    lm_datasets["test"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=32,
)

In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

model_name = model_checkpoint.split("/")[-1]
callback = PushToHubCallback(
    output_dir=f"{model_name}-finetuned-tweet_pemilu_postagger_2", tokenizer=tokenizer
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/freyagracia/indonesian-roberta-base-posp-tagger-finetuned-tweet_pemilu_postagger_2 into local empty directory.


In [ ]:
import math

eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

14/14 [==============================] - 20s 370ms/step - loss: 12.2096
Perplexity: 200702.40


In [ ]:
model.fit(tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback], epochs=10)

Epoch 1/10
31/31 [==============================] - 77s 2s/step - loss: 12.0302 - val_loss: 11.6101


In [ ]:
eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

In [ ]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="freyagracia/indonesian-roberta-base-posp-tagger-finetuned-tweet_pemilu_postagger"
)

config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/655M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForMaskedLM.

All the layers of TFRobertaForMaskedLM were initialized from the model checkpoint at freyagracia/indonesian-roberta-base-posp-tagger-finetuned-tweet_pemilu_postagger.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/808k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/467k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

# MODEL

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd

# Assuming your DataFrame is named df
data['gpt'] = data['gpt'].replace({'negatif': 'NEGATIVE', 'positif': 'POSITIVE'})

# Display the updated DataFrame
data

In [ ]:
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

def retrieve_top_documents(documents, query, model_name='freyagracia/indonesian-roberta-base-posp-tagger-finetuned-tweet_pemilu_postagger', threshold=0.7):
    # Load a pre-trained Hugging Face model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name, from_tf=True)

    # Tokenize and encode documents and query
    document_embeddings = [model(**tokenizer(doc, return_tensors='pt')).last_hidden_state.mean(dim=1).squeeze().detach().numpy() for doc in documents]
    query_embedding = model(**tokenizer(query, return_tensors='pt')).last_hidden_state.mean(dim=1).squeeze().detach().numpy()

    # Compute cosine similarity between the query and each document
    similarities = cosine_similarity([query_embedding], document_embeddings).flatten()

    # Combine document indices and similarity scores into tuples
    result = list(enumerate(similarities))

    # Filter results based on the threshold
    filtered_result = [(idx, score) for idx, score in result if score >= 0.7]

    # Sort the filtered result by similarity score in descending order
    sorted_result = sorted(filtered_result, key=lambda x: x[1], reverse=True)

    return sorted_result

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch

def analyze_sentiment(sentence, model_name="indolem/indobertweet-base-uncased"):
    # Initialize sentiment analysis pipeline for Indonesian
    sentiment_pipeline = pipeline("sentiment-analysis", model="indolem/indobert-base-uncased")

    # Analyze sentiment for the query
    query_sentiment_result = sentiment_pipeline(query)

    # Extract sentiment label
    query_sentiment_label = query_sentiment_result[0]['label']

    # Determine sentiment
    if query_sentiment_label == 'LABEL_0':
        sentiment_category = 'NEGATIVE'
    else:
        sentiment_category = 'POSITIVE'

    return sentiment_category

In [ ]:
# Function to calculate the percentage of relevant documents with the same sentiment as the query
def calculate_matching_sentiment_percentage(query_sentiment, relevant_documents, data):
    # Count the number of relevant documents with the same sentiment as the query
    matching_sentiment_count = sum(1 for idx, _ in relevant_documents if data['gpt'].iloc[idx] == query_sentiment)

    # Calculate the percentage for the same sentiment
    total_relevant_documents = len(relevant_documents)
    percentage_matching_sentiment = (matching_sentiment_count / total_relevant_documents) * 100

    # Calculate the percentage for the opposite sentiment
    percentage_opposing_sentiment = (total_relevant_documents - matching_sentiment_count) / total_relevant_documents * 100

    return percentage_matching_sentiment, percentage_opposing_sentiment


In [ ]:
# Example usage with a threshold of 0.4
threshold = 0.4
query = "calon presiden tidak kompeten"
query = prepro(query)
query = replace_slang(query)
query = remove_url(query)
query = replace_word_elongation(query)
query = str(query)
data["clean"] = data["clean"].astype(str)

# Menggunakan fungsi retrieve_top_documents dengan threshold
relevant_documents = retrieve_top_documents(data["clean"], query, threshold=0.7)

# Sort the relevant_documents list by similarity score in descending order
sorted_documents = sorted(relevant_documents, key=lambda x: x[1], reverse=True)

# Select the top 10 documents
top_documents = sorted_documents[:10]

# Display the top 10 documents
for idx, similarity in top_documents:
    print(f"Document {idx + 1} - Similarity: {similarity:.4f}")
    print(f"   {data['clean'][idx]}")
    print("=" * 50)

# Save the top 10 documents in a variable
top_10_documents = [data["clean"][idx] for idx, _ in top_documents]

All TF 2.0 model weights were used when initializing RobertaModel.

All the weights of RobertaModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaModel for predictions without further training.


Document 1251 - Similarity: 0.8472
   keren banget jawaban pak ganjar bahkan selesai menjawab langsung dapat tepuk tangan meriah dari audience jawaban langsung tepat sasaran diberi fakta hasil kerja beliau memang ganjar paling solutif
Document 5529 - Similarity: 0.8302
   jadi wapres itu tugasnya berat bukan sekedar gunting pita wakilin presiden jika berhalangan hadir tapi juga harus bisa berkontribusi maksimal sudah pak mahfud saja walikota solo biar kembali kerja selesaikan tugas sebagai bentuk tanggung jawab
Document 3565 - Similarity: 0.8260
   gerak cepat untuk ganjar mahfud menang salam jari jangan lupa mulai besok sudah kampanye adapun tahapan tersebut dilaksanakan berdasarkan jadwal sebagai berikut november februari periode ini mencakup pertemuan terbatas pertemuan tatap muka penyebaran bahan
Document 1721 - Similarity: 0.8250
   prof mahfud md jika bapak menjadi wapres nanti saya mohon dengan hormat tolong pak saya jangan di angkat jadi staf bapak lah bapak terlalu jujur saya 

In [ ]:
query='calon presiden tidak kompeten'
query_sentiment = analyze_sentiment(query)
print("Result:", query_sentiment)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Result: NEGATIVE


In [ ]:
# Calculating the percentage of relevant documents with the same and opposite sentiment as the query
matching_sentiment_percentage, opposing_sentiment_percentage = calculate_matching_sentiment_percentage(query_sentiment, relevant_documents, data)

print(f"Percentage of relevant documents with the same sentiment as the query: {matching_sentiment_percentage:.2f}%")
print(f"Percentage of relevant documents with the opposite sentiment as the query: {opposing_sentiment_percentage:.2f}%")

Percentage of relevant documents with the same sentiment as the query: 77.78%
Percentage of relevant documents with the opposite sentiment as the query: 22.22%
